In [1]:
import warnings
warnings.filterwarnings("ignore")
import os, sys
import numpy as np
import pandas as pd
import pickle

In [2]:
#number of bags for stratified bagging
number_of_bags=64

#ADD descpitor column infomation
d1= 1 #starting column number
d2= 27 #ending column number

path= os.getcwd()

In [3]:
#Path to the test file

test_file = os.path.join(path,'test_data/sample_test_file.csv')

In [4]:
dataset_test = pd.read_csv(test_file, delimiter=',')
X_test = dataset_test.iloc[:,d1:d2]

In [5]:
#load model from computer/folder

model_path = os.path.join(path,'SB_models/')
modelList=[]
model_count=0
for i in range (number_of_bags):
        model_name= model_path + 'model_' + str(model_count)
        modelList.append(pickle.load(open(model_name, 'rb')))
        model_count= model_count + 1

In [6]:
# make predictions for test data

#define list that contains all bagging prediction for test set(subsets; 64) 
pred_Bag_savedModel=[]
pred_prob_Bag_savedModel=[]

for i in range (number_of_bags):
    y_pred_prob_savedModel = modelList[i].predict_proba(X_test)[:, 1]
    y_pred_prob_savedModel = np.round(y_pred_prob_savedModel, 2)
    pred_prob_Bag_savedModel.append (y_pred_prob_savedModel)

In [7]:
#Pred Prob list to dataframe
pred_prob_Bag_savedModel_df = pd.DataFrame(pred_prob_Bag_savedModel) 

#Pred Prob list to dataframe--transporsed
pred_prob_Bag_savedModel_df_transposed = pred_prob_Bag_savedModel_df.T 

#Final test data predictions
pred_prob_Bag_savedModel_df_transposed['Probability_of_prediction']= pred_prob_Bag_savedModel_df_transposed.iloc[:,:].mean(axis=1)

# join test prediction to the test set
Final_test_pred_savedModel= pd.concat([dataset_test, pred_prob_Bag_savedModel_df_transposed], axis=1)
Final_test_pred_savedModel.loc[Final_test_pred_savedModel['Probability_of_prediction'] >= 0.5, 'prediction_class'] = 1
Final_test_pred_savedModel.loc[Final_test_pred_savedModel['Probability_of_prediction'] < 0.5, 'prediction_class'] = 0
Final_test_pred_savedModel['prediction_class'] = Final_test_pred_savedModel['prediction_class'].astype(np.int64)

Final_test_pred_savedModel_output = Final_test_pred_savedModel[['Molecule name', 'Probability_of_prediction','prediction_class']].copy()


In [8]:
#Output file name

output_path = os.path.join(path,'output_predictions/')

Final_test_pred_savedModel_output.to_csv(output_path + 'predictions.csv',index=False)
